In [4]:
import pandas as pd
import requests

train_url = "https://raw.githubusercontent.com/yusufsyaifudin/indonesia-ner/master/resources/ner/data_train.txt"
test_url = "https://raw.githubusercontent.com/yusufsyaifudin/indonesia-ner/master/resources/ner/data_test.txt"

# read txt
train = requests.get(train_url).text.splitlines()
test = requests.get(test_url).text.splitlines()

In [21]:
import re

html_tag_pattern = re.compile(r"(<.*?>)(.*?)(</.*?>)")

In [22]:
html_tag_pattern.findall(train[0])

[('<ORGANIZATION>', 'Universitas Gadjah Mada', '</ORGANIZATION>'),
 ('<PERSON>', 'Arie Sudjito', '</PERSON>'),
 ('<ORGANIZATION>', 'Partai Golkar', '</ORGANIZATION>'),
 ('<PERSON>', 'Aburizal Bakrie', '</PERSON>')]

In [24]:
import nltk
from nltk.tokenize import word_tokenize

In [39]:
def create_bio_tags(data):
    new_data = []

    for i in range(len(data)):
        term_with_tags = html_tag_pattern.findall(data[i])
        joined_term_with_tags = [''.join(el) for el in term_with_tags]
        text = data[i]

        for j,joined_term_with_tag in enumerate(joined_term_with_tags):
            text = text.replace(joined_term_with_tag, term_with_tags[j][1])
        
        tokenized_text = word_tokenize(text)
        bio_tags = ['O' for _ in range(len(tokenized_text))]
        for term in term_with_tags:
            if term[0] != '<PERSON>':
                continue
            term = term[1]
            tokenized_term = word_tokenize(term)

            for k in range(0,len(tokenized_text)-len(tokenized_term)+1,1):
                if tokenized_text[k:k+len(tokenized_term)] == tokenized_term:
                    bio_tags[k] = 'B-PERSON'
                    for l in range(1,len(tokenized_term)):
                        bio_tags[k+l] = 'I-PERSON'
            
        new_data.append([tokenized_text,bio_tags])

    return new_data

In [40]:
tokenized_train = create_bio_tags(train)
tokenized_test = create_bio_tags(test)

In [43]:
save_path = "./interim/data.txt"

with open(save_path, "w") as f:
    for data in tokenized_train:
        for token, tag in zip(data[0], data[1]):
            # make token unicode
            token = token.encode('unicode_escape').decode('utf-8')
            f.write(token + "\t" + tag + "\n")
        f.write("\n")
    
    for data in tokenized_test:
        for token, tag in zip(data[0], data[1]):
            # make token unicode
            token = token.encode('unicode_escape').decode('utf-8')
            f.write(token + "\t" + tag + "\n")
        f.write("\n")